## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load ="C:\\Users\\alice\\Desktop\\BootCamp_DataAnaysis\\Practice_and_Challenge\\Module_6_Challenge\\Weather_Database\\WeatherPy_Database.csv"

city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Harper,LR,4.3750,-7.7169,80.02,86,100,6.40,overcast clouds
1,1,Hasaki,JP,35.7333,140.8333,66.00,39,20,14.97,few clouds
2,2,São João da Barra,BR,-21.6403,-41.0511,67.44,88,0,9.19,clear sky
3,3,Paka,MY,4.6374,103.4368,86.00,74,20,3.44,few clouds
4,4,Castro,BR,-24.7911,-50.0119,51.08,95,22,2.46,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Harper,LR,4.3750,-7.7169,80.02,86,100,6.40,overcast clouds
3,3,Paka,MY,4.6374,103.4368,86.00,74,20,3.44,few clouds
5,5,Puerto Ayora,EC,-0.7393,-90.3518,77.00,95,44,3.00,scattered clouds
6,6,Kapaa,US,22.0752,-159.3190,78.80,69,40,8.05,scattered clouds
7,7,Avarua,CK,-21.2078,-159.7750,78.80,78,75,12.66,broken clouds
8,8,Rikitea,PF,-23.1203,-134.9692,77.92,78,100,14.05,light rain
9,9,Bengkulu,ID,-3.8004,102.2655,83.01,63,63,4.38,broken clouds
14,14,Vaini,TO,-21.2000,-175.2000,86.00,79,75,4.61,light rain
16,16,Touros,BR,-5.1989,-35.4608,77.00,88,75,4.61,broken clouds
31,31,Kavieng,PG,-2.5744,150.7967,82.29,76,90,7.78,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how="any")
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Harper,LR,4.3750,-7.7169,80.02,86,100,6.40,overcast clouds
3,3,Paka,MY,4.6374,103.4368,86.00,74,20,3.44,few clouds
5,5,Puerto Ayora,EC,-0.7393,-90.3518,77.00,95,44,3.00,scattered clouds
6,6,Kapaa,US,22.0752,-159.3190,78.80,69,40,8.05,scattered clouds
7,7,Avarua,CK,-21.2078,-159.7750,78.80,78,75,12.66,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Harper,LR,80.02,overcast clouds,4.3750,-7.7169,
3,Paka,MY,86.00,few clouds,4.6374,103.4368,
5,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,
6,Kapaa,US,78.80,scattered clouds,22.0752,-159.3190,
7,Avarua,CK,78.80,broken clouds,-21.2078,-159.7750,
8,Rikitea,PF,77.92,light rain,-23.1203,-134.9692,
9,Bengkulu,ID,83.01,broken clouds,-3.8004,102.2655,
14,Vaini,TO,86.00,light rain,-21.2000,-175.2000,
16,Touros,BR,77.00,broken clouds,-5.1989,-35.4608,
31,Kavieng,PG,82.29,light rain,-2.5744,150.7967,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Harper,LR,80.02,overcast clouds,4.3750,-7.7169,Screensaver
3,Paka,MY,86.00,few clouds,4.6374,103.4368,RUMBIA RESORT VILLA
5,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Kapaa,US,78.80,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
7,Avarua,CK,78.80,broken clouds,-21.2078,-159.7750,Paradise Inn
8,Rikitea,PF,77.92,light rain,-23.1203,-134.9692,Pension Maro'i
9,Bengkulu,ID,83.01,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
14,Vaini,TO,86.00,light rain,-21.2000,-175.2000,Keleti Beach Resort
16,Touros,BR,77.00,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
31,Kavieng,PG,82.29,light rain,-2.5744,150.7967,Nusa Island Retreat


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head(20)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Harper,LR,80.02,overcast clouds,4.3750,-7.7169,Screensaver
3,Paka,MY,86.00,few clouds,4.6374,103.4368,RUMBIA RESORT VILLA
5,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Kapaa,US,78.80,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
7,Avarua,CK,78.80,broken clouds,-21.2078,-159.7750,Paradise Inn
8,Rikitea,PF,77.92,light rain,-23.1203,-134.9692,Pension Maro'i
9,Bengkulu,ID,83.01,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
14,Vaini,TO,86.00,light rain,-21.2000,-175.2000,Keleti Beach Resort
16,Touros,BR,77.00,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
31,Kavieng,PG,82.29,light rain,-2.5744,150.7967,Nusa Island Retreat


In [11]:
clean_hotel_df.count()

City                   185
Country                185
Max Temp               185
Current Description    185
Lat                    185
Lng                    185
Hotel Name             185
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
locations.head()

,Lat,Lng
0,4.3750,-7.7169
3,4.6374,103.4368
5,-0.7393,-90.3518
6,22.0752,-159.3190
7,-21.2078,-159.7750


In [15]:
# print(hotel_info)

In [16]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))